In [28]:
import os
import sys
sys.path.append('/Users/aghavamp/Desktop/Projects')
sys.path.append('/Users/aghavamp/Desktop/Projects/Functional_Fusion')
sys.path.append('/Users/aghavamp/Desktop/Projects/SUITPy')
import getpass
import importlib
import tqdm

import scipy.io as sio
import rsatoolbox as rsa
from rsatoolbox.io import spm as spm_io
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import surfAnalysisPy as surf
import SUITPy as suit
import nibabel as nb
import nitools as nt
from matplotlib.cm import ScalarMappable
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from pathlib import Path
import seaborn as sns
import PcmPy as pcm
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel
import glob
import matplotlib.patches as patches
from nilearn.plotting.cm import _cmap_d as nilearn_cmaps

# SET PATHS:
baseDir = os.path.join('/Users', getpass.getuser(), 'Desktop', 'Projects', 'bimanual_wrist', 'data', 'fMRI')
bidsDir = 'BIDS'
anatomicalDir = 'anatomicals'
freesurferDir = 'surfaceFreesurfer'
surfacewbDir = 'surfaceWB' 
behavDir = 'behavioural'
regDir = 'ROI'
atlasDir = '/Volumes/diedrichsen_data$/data/Atlas_templates/fs_LR_32'
analysisDir = os.path.join(os.path.dirname(os.path.dirname(baseDir)), 'analysis')




# wb view

In [ ]:
import subprocess
surfPy_dir = '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh'
atlas_temp_dir = '/Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/FS_LR_32'

# add paths:
paths = []
# add surface files:
# paths.append(os.path.abspath(os.path.join(surfPy_dir, f'fs_L', f'fs_LR.32k.Lm.flat.surf.gii')))
# paths.append(os.path.abspath(os.path.join(surfPy_dir, f'fs_R', f'fs_LR.32k.R.flat.surf.gii')))
paths.append(os.path.abspath(os.path.join(surfPy_dir, f'fs_L', f'fs_LR.32k.LR.sulc.dscalar.nii')))
# paths.append(os.path.abspath(os.path.join(surfPy_dir, f'fs_L', f'fs_LR.32k.L.shape.gii')))
paths.append(os.path.abspath(os.path.join(atlas_temp_dir, 'fs_LR.32k.L.inflated.surf.gii')))
paths.append(os.path.abspath(os.path.join(atlas_temp_dir, 'fs_LR.32k.R.inflated.surf.gii')))

paths.append(os.path.abspath(os.path.join(atlas_temp_dir, 'ROI.32k.L.label.gii')))
paths.append(os.path.abspath(os.path.join(atlas_temp_dir, 'ROI.32k.R.label.gii')))
borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
           'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}
paths.append(borders['L'])
paths.append(borders['R'])

# add psc:
paths.append(os.path.abspath(os.path.join(analysisDir, f'psc.dscalar.nii')))

# Run wb_view:
subprocess.run(['wb_view', *paths])




Info: Resources loaded:
   :/About   :/BalsaUploadDialog   :/Cursor   :/DingOntology   :/Fonts   :/general_resources.qrc   :/help_resources.qrc   :/HelpFiles   :/LayersPanel   :/MessageDialog   :/PaletteEditorDialog   :/PaletteSettings   :/qpdf   :/qt-project.org   :/RecentFilesDialog   :/SceneFileDialog   :/SpecFileDialog   :/Splash   :/ToolBar   :/update_resources.sh


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/FS_LR_32/fs_LR.32k.L.inflated.surf.gii was 0.02816 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/FS_LR_32/fs_LR.32k.R.inflated.surf.gii was 0.022806 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.LR.sulc.dscalar.nii was 0.023422 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/FS_LR_32/ROI.32k.L.label.gii was 0.000527 seconds.


Info: Time to read /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI

# surface analysis

In [26]:
conds = ['lhand', 'rhand', 'bi']
BOLD_threshold = 0.2
fwhm_mm = 3 # to know which file to load

# Load the PSC cifti file:
psc_cifti_path = os.path.join(analysisDir, f'psc_{fwhm_mm}mm.dscalar.nii')
psc_cifti = nb.load(psc_cifti_path)
subject_data_array = psc_cifti.get_fdata()
scalar_axis = psc_cifti.header.get_axis(0)
combined_axis = psc_cifti.header.get_axis(1)

n_vertices_left = combined_axis.nvertices['CIFTI_STRUCTURE_CORTEX_LEFT']
n_vertices_right = combined_axis.nvertices['CIFTI_STRUCTURE_CORTEX_RIGHT']

# group level t-statistics:
combined_data = subject_data_array
for cond in conds:
    idx = [i for i, label in enumerate(scalar_axis.name) if cond in label]
    # Extract the data for the current condition
    condition_data = subject_data_array[idx, :]
    
    mean_data = np.nanmean(condition_data, axis=0, keepdims=True)
    std_data = np.nanstd(condition_data, axis=0, keepdims=True, ddof=1)
    DoF = np.sum(~np.isnan(condition_data), axis=0, keepdims=True) - 1
    tstats = mean_data / (std_data / np.sqrt(DoF+1))
    tstats[~np.isfinite(tstats)] = np.nan # handle voxels with no data
    
    # mean_data significant max at abs(tstat) > 1.8:
    p05_mask = np.abs(tstats) > 1.8
    mean_data_significant = mean_data * p05_mask

    # mean data thresholded:
    mean_data_thresholded = mean_data.copy()
    mean_data_thresholded[np.abs(mean_data) < BOLD_threshold] = 0
    
    # add to the data:
    combined_data = np.concatenate([combined_data, mean_data, mean_data_significant, mean_data_thresholded, tstats, DoF], axis=0)
    
    # update the scalar axis:
    scalar_axis = scalar_axis + nb.cifti2.cifti2_axes.ScalarAxis(name=[f'mean_psc_{cond}'])
    scalar_axis = scalar_axis + nb.cifti2.cifti2_axes.ScalarAxis(name=[f'mean_psc_p05_{cond}'])
    scalar_axis = scalar_axis + nb.cifti2.cifti2_axes.ScalarAxis(name=[f'mean_psc_thresholded_{cond}'])
    scalar_axis = scalar_axis + nb.cifti2.cifti2_axes.ScalarAxis(name=[f'tstat_{cond}'])
    scalar_axis = scalar_axis + nb.cifti2.cifti2_axes.ScalarAxis(name=[f'DoF_{cond}'])

# get the contrast contra - ipsi:
lhand_mean_idx = scalar_axis.name.tolist().index('mean_psc_lhand')
rhand_mean_idx = scalar_axis.name.tolist().index('mean_psc_rhand')
lhand_data = combined_data[lhand_mean_idx, :]
rhand_data = combined_data[rhand_mean_idx, :]
contras_contra_ipsi = np.zeros_like(lhand_data)
contras_contra_ipsi[:n_vertices_left] = rhand_data[:n_vertices_left] - lhand_data[:n_vertices_left]
contras_contra_ipsi[n_vertices_left:] = lhand_data[n_vertices_left:] - rhand_data[n_vertices_left:]
# threshold:
contras_contra_ipsi_thresholded = contras_contra_ipsi.copy()
contras_contra_ipsi_thresholded[np.abs(contras_contra_ipsi_thresholded) < BOLD_threshold] = 0

combined_data = np.concatenate([combined_data, contras_contra_ipsi[np.newaxis, :], contras_contra_ipsi_thresholded[np.newaxis, :]], axis=0)
scalar_axis = scalar_axis + nb.cifti2.cifti2_axes.ScalarAxis(name=[f'contra_minus_ipsi'])
scalar_axis = scalar_axis + nb.cifti2.cifti2_axes.ScalarAxis(name=[f'contra_minus_ipsi_thresholded'])

# Create the full CIFTI header
header = nb.cifti2.Cifti2Header.from_axes([scalar_axis, combined_axis])

# Create the Cifti2Image object
cifti_img = nb.cifti2.Cifti2Image(combined_data, header)

nb.save(cifti_img, os.path.join(analysisDir, f'psc.dscalar.nii'))



/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_96406/2085090367.py:22: RuntimeWarning: Mean of empty slice
  mean_data = np.nanmean(condition_data, axis=0, keepdims=True)
/Users/aghavamp/Desktop/Projects/bimanual_wrist/.conda/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_96406/2085090367.py:25: RuntimeWarning: invalid value encountered in divide
  tstats = mean_data / (std_data / np.sqrt(DoF+1))


# calcuate Surface PSC

Projecting the average contrast for conditions rhand,lhand, and bimanual to the surface and saving as cifti

In [ ]:
import subprocess
import numpy as np

fwhm_mm = 3

def smooth_cifti(input_cifti, output_cifti, left_surf, right_surf, fwhm_mm):
    """
    Smooths a dscalar CIFTI file using wb_command.
    """
    # Convert FWHM to Sigma
    sigma = fwhm_mm / 2.355
    sigma_str = f"{sigma:.2f}"

    cmd = [
        "wb_command", 
        "-cifti-smoothing",
        input_cifti,
        sigma_str,  # Surface smoothing kernel
        sigma_str,  # Volume smoothing kernel (even if no volume data, this is required)
        "COLUMN",   # Smoothing direction for dscalar
        output_cifti,
        "-left-surface", left_surf,
        "-right-surface", right_surf
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"Successfully created: {output_cifti}")
    except subprocess.CalledProcessError as e:
        print(f"Error running wb_command: {e}")

glm = 1
conds = ['lhand', 'rhand', 'bi']
sn_list = [101,104,106,107,108,109,110,111,112,113,114,115]
BOLD_threshold = 0.2
all_data = []
scalar_labels = []

for sn in sn_list:
    # Define the relevant gifti files for both left and right hemisphere:
    surf_white = [] # White -gray matter surface 
    surf_pial = []  # Pial surface
    hemN = ['L','R']
    for h,hem in enumerate(hemN):
        surf_white.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.white.32k.surf.gii'))
        surf_pial.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.pial.32k.surf.gii'))

    for cond in conds:
        datafiles = [os.path.join(baseDir, f'glm{glm}', f's{sn}', f'PSC_{cond}.nii')]
        
        # Map a Nifti to the surface of left hemisphere
        DL = surf.map.vol_to_surf(datafiles,surf_pial[0], surf_white[0])
        # if DL.shape[1] > 1:
        #     DL = DL[:,0] / DL[:,1] * 100 # convert to percent signal change
        # else:
        #     print('WARNING: Constant not found?')
        # Map a Nifti to the surface of right hemisphere
        DR = surf.map.vol_to_surf(datafiles,surf_pial[1],surf_white[1])
        # if DR.shape[1] > 1:
        #     DR = DR[:,0] / DR[:,1] * 100 # convert to percent signal change

        D = np.concatenate([DL, DR])

        all_data.append(D.flatten())
        scalar_labels.append(f's{sn}_con_{cond}')

# make BrainModelAxis:
n_vertices_left = len(DL)
n_vertices_right = len(DR)

# Create a boolean mask for the left hemisphere (all True)
mask_left = np.ones(n_vertices_left, dtype=bool)
bm_left = nb.cifti2.BrainModelAxis.from_mask(mask_left, name='CIFTI_STRUCTURE_CORTEX_LEFT')

# Create a boolean mask for the right hemisphere (all True)
mask_right = np.ones(n_vertices_right, dtype=bool)
bm_right = nb.cifti2.BrainModelAxis.from_mask(mask_right, name='CIFTI_STRUCTURE_CORTEX_RIGHT')

# Combine the left and right axes
combined_axis = bm_left + bm_right

# stack all data into a single array
final_data_array = np.vstack(all_data)
# Create the scalar axis from your list of all labels
scalar_axis = nb.cifti2.ScalarAxis(scalar_labels)

# Create the full CIFTI header
header = nb.cifti2.Cifti2Header.from_axes([scalar_axis, combined_axis])
# Create the Cifti2Image object
cifti_img = nb.cifti2.Cifti2Image(final_data_array, header)
nb.save(cifti_img, os.path.join(analysisDir, f'psc_nonsmooth.dscalar.nii'))


# ==========================================
# =============== SMOOTHING ================
if fwhm_mm > 0:
    left_surf = os.path.abspath(os.path.join(atlas_temp_dir, 'fs_LR.32k.L.midthickness.surf.gii'))
    right_surf = os.path.abspath(os.path.join(atlas_temp_dir, 'fs_LR.32k.R.midthickness.surf.gii'))
    
    input_cifti = os.path.abspath(os.path.join(analysisDir, f'psc_nonsmooth.dscalar.nii'))
    output_cifti = os.path.abspath(os.path.join(analysisDir, f'psc_{fwhm_mm}mm.dscalar.nii'))
    
    smooth_cifti(
        input_cifti=input_cifti,
        output_cifti=output_cifti,
        left_surf=left_surf,
        right_surf=right_surf,
        fwhm_mm=fwhm_mm
    )




Successfully created: /Users/aghavamp/Desktop/Projects/bimanual_wrist/analysis/psc_3mm.dscalar.nii


# Volume smoothing

smooth the contrasts in volume with certain fwhm_mm

In [10]:
import os, glob
from nilearn import image
import nibabel as nb

glm = 1
conds = ['lhand', 'rhand', 'bi']
sn_list = [101,102,103,104,106,107,108,109,110,111,112,113,114,115]
# sn_list = [101,102,103,104,106]
all_data = []
scalar_labels = []

fwhm_mm = 3 # in mm
for sn in sn_list:
    for cond in conds:
        # datafile = [os.path.join(baseDir, f'glm{glm}', f's{sn}', f'con_{cond}.nii')]
        datafile = [os.path.join(baseDir, f'glm{glm}', f's{sn}', f'PSC_{cond}.nii')]
        sm = image.smooth_img(datafile, fwhm=fwhm_mm)
        # out = os.path.join(baseDir, f'glm{glm}', f's{sn}', f"con_{cond}_smooth{fwhm_mm}mm.nii")
        out = os.path.join(baseDir, f'glm{glm}', f's{sn}', f"PSC_{cond}_smooth{fwhm_mm}mm.nii")
        nb.save(sm[0], out)
        print("Saved:", out)



Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s101/PSC_lhand_smooth3mm.nii
Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s101/PSC_rhand_smooth3mm.nii
Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s101/PSC_bi_smooth3mm.nii
Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s102/PSC_lhand_smooth3mm.nii
Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s102/PSC_rhand_smooth3mm.nii
Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s102/PSC_bi_smooth3mm.nii
Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s103/PSC_lhand_smooth3mm.nii
Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s103/PSC_rhand_smooth3mm.nii
Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s103/PSC_bi_smooth3mm.nii
Saved: /Users/aghavamp/Desktop/Projects/bimanual_wrist/data/fMRI/glm1/s104/PSC_lhand_smooth3mm.nii
Saved: /Users/aghav

## Plot on surface

In [2]:
glm = 1
participants = [101,102,103,104,106,107,108,109,110,111,112,113,114,115]
cmap = 'jet'
tmax = 12

for _, sn in enumerate(participants):
    # Define the relevant gifti files for both left and right hemisphere:
    surf_white = [] # White -gray matter surface 
    surf_pial = []  # Pial surface
    surf_sulc = []  # Sulcal depth 
    hemN = ['L','R']
    for h,hem in enumerate(hemN):
        surf_white.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.white.32k.surf.gii'))
        surf_pial.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.pial.32k.surf.gii'))
        surf_sulc.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.sulc.32k.shape.gii'))
    
    spmT_names = [os.path.join(baseDir, f'glm{glm}', f's{sn}', 'spmT_lhand.nii'), 
                  os.path.join(baseDir, f'glm{glm}', f's{sn}', 'spmT_rhand.nii'),
                  os.path.join(baseDir, f'glm{glm}', f's{sn}', 'spmT_bi.nii')]
    for spmT in spmT_names:
        # Map a Nifti to the surface of left hemisphere
        DL = surf.map.vol_to_surf([spmT],surf_pial[0], surf_white[0])
        # Map a Nifti to the surface of right hemisphere
        DR = surf.map.vol_to_surf([spmT],surf_pial[1],surf_white[1])

        D = {'L':DL, 'R':DR}

        Hem = ['L', 'R']
        region_names = ['?', 'S1', 'M1', 'PMd', 'PMv', 'SMA', 'V1', 'SPLa', '?']
        borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
                   'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}
        
        # Combine data from both hemispheres
        all_data = np.concatenate([D['L'].flatten(), D['R'].flatten()])
        # Use percentiles
        # vmin = np.nanpercentile(all_data,20)
        vmax = tmax
        vmin = -vmax
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for ax, H in zip(axs, Hem):
            plt.sca(ax)
            surf.plot.plotmap(D[H].mean(axis=1), f'fs32k_{H}',
                                underlay=None,
                                borders=borders[H],
                                cscale=[vmin, vmax],
                                cmap=cmap,
                                underscale=[-1.5, 1],
                                alpha=.7,
                                new_figure=False,
                                colorbar=False,
                                # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
                                )
        # make colorbar
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        sm = ScalarMappable(norm=norm, cmap=cmap)
        cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
        cbar.set_label('contrast')

        axs[0].set_title('Left hemisphere')
        axs[1].set_title('Right hemisphere')
        fig.suptitle(f'contrast, s{sn}, glm{glm}, {spmT}')
        fig.tight_layout()
        plt.savefig(f'../figures/tmaps/s{sn}_{spmT.split("/")[-1].split(".")[0]}.pdf', bbox_inches="tight")
        plt.close()



/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_15851/1770452856.py:63: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_15851/1770452856.py:63: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_15851/1770452856.py:63: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_15851/1770452856.py:63: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_15851/1770452856.py:63: UserWarning: This figure includes Axes that are n

## Plot ResMS on surface

In [12]:
glm = 1
participants = [101,102,103,104,106,107,108,109,110,111,112,113,114,115]
cmap = 'coolwarm'
tmax = 10

for _, sn in enumerate(participants):
    # Define the relevant gifti files for both left and right hemisphere:
    surf_white = [] # White -gray matter surface 
    surf_pial = []  # Pial surface
    surf_sulc = []  # Sulcal depth 
    hemN = ['L','R']
    for h,hem in enumerate(hemN):
        surf_white.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.white.32k.surf.gii'))
        surf_pial.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.pial.32k.surf.gii'))
        surf_sulc.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.sulc.32k.shape.gii'))
    
    spmT_names = sorted(glob.glob(os.path.join(baseDir, f'glm{glm}', f's{sn}', "ResMS.nii")))
    for spmT in spmT_names:
        # Map a Nifti to the surface of left hemisphere
        DL = surf.map.vol_to_surf([spmT],surf_pial[0], surf_white[0])
        # Map a Nifti to the surface of right hemisphere
        DR = surf.map.vol_to_surf([spmT],surf_pial[1],surf_white[1])

        D = {'L':np.sqrt(DL), 'R':np.sqrt(DR)}

        Hem = ['L', 'R']
        borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
                   'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}
        
        # Combine data from both hemispheres
        all_data = np.concatenate([D['L'].flatten(), D['R'].flatten()])
        # Use percentiles
        # vmin = np.nanpercentile(all_data,20)
        vmax = tmax
        vmin = 0
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for ax, H in zip(axs, Hem):
            plt.sca(ax)
            surf.plot.plotmap(D[H].mean(axis=1), f'fs32k_{H}',
                                underlay=None,
                                borders=borders[H],
                                cscale=[vmin, vmax],
                                cmap=cmap,
                                underscale=[-1.5, 1],
                                alpha=.5,
                                new_figure=False,
                                colorbar=False,
                                # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
                                )
        # make colorbar
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        sm = ScalarMappable(norm=norm, cmap=cmap)
        cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
        cbar.set_label('sqrt(ResMS)')

        axs[0].set_title('Left hemisphere')
        axs[1].set_title('Right hemisphere')
        fig.suptitle(f'R2, s{sn}, glm{glm}, {spmT}')
        fig.tight_layout()
        plt.savefig(f'../figures/ResMS/s{sn}_ResMS_glm{glm}.pdf', bbox_inches="tight")
        plt.close()
        
    # D = {'L':DL_avg, 'R':DR_avg}
    # all_data = np.concatenate([DL_avg.flatten(), DR_avg.flatten()])
    # # Use percentiles
    # # vmin = np.nanpercentile(all_data,20)
    # vmax = np.nanpercentile(all_data, tmax)
    # vmin = -vmax
    # fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    # for ax, H in zip(axs, Hem):
    #     plt.sca(ax)
    #     surf.plot.plotmap(D[H].mean(axis=1), f'fs32k_{H}',
    #                         underlay=None,
    #                         borders=borders[H],
    #                         cscale=[vmin, vmax],
    #                         cmap=cmap,
    #                         underscale=[-1.5, 1],
    #                         alpha=.5,
    #                         new_figure=False,
    #                         colorbar=False,
    #                         # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
    #                         )
    # # make colorbar
    # norm = plt.Normalize(vmin=vmin, vmax=vmax)
    # sm = ScalarMappable(norm=norm, cmap=cmap)
    # cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
    # cbar.set_label('t-value')

    # axs[0].set_title('Left hemisphere')
    # axs[1].set_title('Right hemisphere')
    # fig.suptitle(f'avg, glm{glm}, {spmT.split("/")[-1].split(".")[0]}')
    # fig.tight_layout()
    # plt.savefig(f'../figures/glmsingle/tmaps/avg_glm{glm}_{spmT.split("/")[-1].split(".")[0]}.pdf', bbox_inches="tight")



/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_8827/3405301150.py:60: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_8827/3405301150.py:60: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_8827/3405301150.py:60: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_8827/3405301150.py:60: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_8827/3405301150.py:60: UserWarning: This figure includes Axes that are not co

## mask.nii on surface

In [ ]:
glm = 'single'
participants = [101,102,103,104,106,107,108,109,110,111,112,113,114,115]
# participants = [113]
cmap = 'jet'
tmax = 1

for _, sn in enumerate(participants):
    # Define the relevant gifti files for both left and right hemisphere:
    surf_white = [] # White -gray matter surface 
    surf_pial = []  # Pial surface
    surf_sulc = []  # Sulcal depth 
    hemN = ['L','R']
    for h,hem in enumerate(hemN):
        surf_white.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.white.32k.surf.gii'))
        surf_pial.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.pial.32k.surf.gii'))
        surf_sulc.append(os.path.join(baseDir,surfacewbDir, f's{sn}', f's{sn}.{hem}.sulc.32k.shape.gii'))
    
    spmT_names = sorted(glob.glob(os.path.join(baseDir, f'glm{glm}', f's{sn}', "mask.nii")))
    for spmT in spmT_names:
        # Map a Nifti to the surface of left hemisphere
        DL = surf.map.vol_to_surf([spmT],surf_pial[0], surf_white[0])
        # Map a Nifti to the surface of right hemisphere
        DR = surf.map.vol_to_surf([spmT],surf_pial[1],surf_white[1])

        D = {'L':DL, 'R':DR}

        Hem = ['L', 'R']
        region_names = ['?', 'S1', 'M1', 'PMd', 'PMv', 'SMA', 'V1', 'SPLa', '?']
        borders = {'L': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_L/fs_LR.32k.L.border',
                   'R': '/Users/aghavamp/Desktop/Projects/surfAnalysisPy/standard_mesh/fs_R/fs_LR.32k.R.border'}
        
        # Combine data from both hemispheres
        all_data = np.concatenate([D['L'].flatten(), D['R'].flatten()])
        # Use percentiles
        # vmin = np.nanpercentile(all_data,20)
        vmax = tmax
        vmin = 0
        
        fig, axs = plt.subplots(1, 2, figsize=(12, 6))
        for ax, H in zip(axs, Hem):
            plt.sca(ax)
            surf.plot.plotmap(D[H].mean(axis=1), f'fs32k_{H}',
                                underlay=None,
                                borders=borders[H],
                                cscale=[vmin, vmax],
                                cmap=cmap,
                                underscale=[-1.5, 1],
                                alpha=.5,
                                new_figure=False,
                                colorbar=False,
                                # frame=[xlim[0], xlim[1], ylim[0], ylim[1]]
                                )
        # make colorbar
        norm = plt.Normalize(vmin=vmin, vmax=vmax)
        sm = ScalarMappable(norm=norm, cmap='jet')
        cbar = fig.colorbar(sm, ax=[axs[0], axs[1]], orientation='horizontal', fraction=0.03)
        cbar.set_label('R-squared percentage')
        
        axs[0].set_title('Left hemisphere')
        axs[1].set_title('Right hemisphere')
        fig.suptitle(f'glm mask, s{sn}, glm{glm}, {spmT}')
        fig.tight_layout()
        plt.savefig(f'../figures/glmsingle/tmaps/glmmask_s{sn}.pdf', bbox_inches="tight")
        plt.close()
  


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1076e5dd0>>
Traceback (most recent call last):
  File "/Users/aghavamp/Desktop/Projects/bimanual_wrist/.conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
/var/folders/sc/kmc7_9gn2hqd_lrnhdt_2s580000gn/T/ipykernel_74181/808450440.py:62: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()
